In [2]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 12462, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 12462 (delta 0), reused 3 (delta 0), pack-reused 12459
Receiving objects: 100% (12462/12462), 11.59 MiB | 27.60 MiB/s, done.
Resolving deltas: 100% (8671/8671), done.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
     |████████████████████████████████| 596 kB 17.7 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
#알약 인식 후 해당 부분 크롭하여 저장
#pill_yolo.pt파일 필요
from IPython.display import Image
import os
from glob import glob

img_list = glob('/content/알약/장방알약.jpg')

print(len(img_list))

val_img_path = img_list[0]

weights_path = '/content/pill_yolo.pt'

!python detect.py --weights "{weights_path}" --img 416 --conf 0.5 --source "{val_img_path}" --line-thickness 1 --save-crop

detect_img_path = '/content/알약/장방알약.jpg'
Image(os.path.join(detect_img_path, os.path.basename(val_img_path)))

1
detect: weights=['/content/pill_yolo.pt'], source=/content/알약/장방알약.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-97-g2da6866 torch 1.10.0+cu111 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/알약/장방알약.jpg: 320x416 1 pill, Done. (0.136s)
Speed: 0.9ms pre-process, 135.7ms inference, 13.1ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp


In [9]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import matplotlib.pyplot as plt

def setLabel(img, pts, label): #비율 계산에 사용되는 레이블 함수
    (x, y, w, h) = cv2.boundingRect(pts)
    pt1 = (x, y)
    pt2 = (x + w, y + h)
    cv2.rectangle(img, pt1, pt2, (0, 255, 0), 2)
    cv2.putText(img, label, (pt1[0], pt1[1]-3), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255))

#def remove(path) #배경제거 함수

def binary(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, dst = cv2.threshold (gray, 0, 255, cv2.THRESH_BINARY)
    cv2.imwrite("/content/binary.png", dst)
    return '/content/binary.png'

def reverse(path): #이진화된 이미지 반전시키는 함수
    img = cv2.imread(path)
    out = img.copy()
    out = 255 - out
    cv2.imwrite("/content/pill.png", out)
    return '/content/pill.png'

def cont(path): #원, 타원, 장방형의 각형을 알려주는 함수
    img = cv2.imread(path)
    img2 = img.copy()
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res, thr = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    cnt = contours[0]
    cv2.drawContours(img, [cnt], -1, (255, 255, 0), 2)

    epsilon2 =  0.005 * cv2.arcLength(cnt, True)
    approx2 = cv2.approxPolyDP(cnt, epsilon2, True)
    cv2.drawContours(img2, [approx2], -1, (0, 255, 0), 3)

    if len(approx2) == 16: #16각형이면 detect함수 실행
        detect(path)
    else: #16각형이 아니면 장방형
        print("장방형")

def detect(path): #가로세로 비율을 통해 원인지 타원인지 분류하는 함수
    src = cv2.imread(reverse(path))
    dst = src.copy()

    gray = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
    ret, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

    contours, hierarchy = cv2.findContours(binary, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

    for i in contours:
        M = cv2.moments(i)
        cX = int(M['m10'] / M['m00'])
        cY = int(M['m01'] / M['m00'])
        
        cv2.circle(dst, (cX, cY), 3, (0, 0, 255), -1)
        cv2.drawContours(dst, [i], 0, (0, 0, 255), 2)

        setLabel(dst, i, '')
        (x, y, w, h) = cv2.boundingRect(i)

        if (h/w) > 0.95 and (h/w) < 1.05: #비율이 0.95와 1.05사이이면 원 출력
          print("원형")
        else: #그렇지 않으면 타원 출력
          print("타원형")

def main(): #메인함수
    #rm_path = remove() #배경제거 함수 호출
    b_path = binary('/content/배경제거/캡슐알약_장방_-removebg-preview.png')
    cont(b_path)

if __name__ == "__main__":
	  main()

장방형
